In [54]:
import pandas as pd
import numpy as np

In [145]:
orig_data = pd.read_csv("master.csv")
print(orig_data.shape)
orig_data.head()

(2215, 211)


,Unnamed: 0,Product,Measures,Week Ending 01-03-16,Week Ending 01-03-16.1,Week Ending 01-03-16.2,Week Ending 01-03-16.3,Week Ending 01-10-16,Week Ending 01-10-16.1,Week Ending 01-10-16.2,...,Week Ending 12-11-16.2,Week Ending 12-11-16.3,Week Ending 12-18-16,Week Ending 12-18-16.1,Week Ending 12-18-16.2,Week Ending 12-18-16.3,Week Ending 12-25-16,Week Ending 12-25-16.1,Week Ending 12-25-16.2,Week Ending 12-25-16.3
0,0,Product,Measures,Current,Year Ago,2 Years Ago,3 Years Ago,Current,Year Ago,2 Years Ago,...,2 Years Ago,3 Years Ago,Current,Year Ago,2 Years Ago,3 Years Ago,Current,Year Ago,2 Years Ago,3 Years Ago
1,1,BUD LIGHT 36-PACK 12 OZ CAN,Price per Unit,21.9895690398207,21.1738502549875,20.2244609105822,21.7891725788077,22.8732126552712,21.9907922192799,20.5934128959276,...,22.5574741607116,20.5941535829129,20.2207810491128,22.0907732784259,21.8739374341181,21.9915715286897,19.5084296740994,21.9911619820341,20.9167181777879,21.6445701550752
2,2,BUD LIGHT 36-PACK 12 OZ CAN,Price per Volume,14.6597126932138,14.1159001699917,13.4829739403881,14.5261150525384,15.2488084368475,14.6605281461866,13.7289419306184,...,15.0383161071411,13.7294357219419,13.4805206994085,14.7271821856173,14.5826249560787,14.6610476857931,13.0056197827329,14.6607746546894,13.9444787851919,14.4297134367168
3,3,BUD LIGHT 36-PACK 12 OZ CAN,Unit Sales,20684.8231011629,23677.2859435081,28310.5796599388,23960.5929021835,11179.2787908315,14677.1865057945,20573.2441139221,...,15693.6328786611,22202.6579779386,17774.021343708,15610.1341928243,17795.9037729501,27935.932192564,26638.1554508209,20512.5194001197,25609.2649072408,33616.9600862264
4,4,BUD LIGHT 36-PACK 12 OZ CAN,Volume Sales,31027.2346517443,35515.9289152622,42465.8694899082,35940.8893532753,16768.9181862473,22015.7797586917,30859.8661708831,...,23540.4493179917,33303.9869669079,26661.032015562,23415.2012892365,26693.8556594252,41903.898288846,39957.2331762313,30768.7791001796,38413.8973608613,50425.4401293396


In [146]:
cols = orig_data.columns
weeks = orig_data[3:]

new_df = pd.melt(orig_data, id_vars=cols[:3], var_name="Date", value_name="Value")
new_df.head()

,Unnamed: 0,Product,Measures,Date,Value
0,0,Product,Measures,Week Ending 01-03-16,Current
1,1,BUD LIGHT 36-PACK 12 OZ CAN,Price per Unit,Week Ending 01-03-16,21.9895690398207
2,2,BUD LIGHT 36-PACK 12 OZ CAN,Price per Volume,Week Ending 01-03-16,14.6597126932138
3,3,BUD LIGHT 36-PACK 12 OZ CAN,Unit Sales,Week Ending 01-03-16,20684.8231011629
4,4,BUD LIGHT 36-PACK 12 OZ CAN,Volume Sales,Week Ending 01-03-16,31027.2346517443


In [147]:
temp = pd.concat([new_df, pd.DataFrame()])
temp = temp.sort_values('Date')

temp = temp[temp["Measures"] != "Measures"]
temp.head()

for i, row in temp.iterrows():
    date = row["Date"]
    date = date.split(" ")[-1]
    if date.endswith("16.1"):
        date = date.replace("16.1", "15")
    if date.endswith("16.2"):
        date = date.replace("16.2", "14")
    if date.endswith("16.3"):
        date = date.replace("16.3", "13")
    temp.set_value(i, "Date", date)

temp.head()

,Unnamed: 0,Product,Measures,Date,Value
1485,1485,TECATE SINGLES 24 OZ CAN,Unit Sales,01-03-16,6175.85378253459
1484,1484,TECATE SINGLES 24 OZ CAN,Price per Volume,01-03-16,24.0189786195096
1483,1483,TECATE SINGLES 24 OZ CAN,Price per Unit,01-03-16,2.00078091900515
1482,1482,MILLER LITE 6-PACK 11.2-13 OZ GLASS,Category Weighted Distribution,01-03-16,72.5016769271537
1481,1481,MILLER LITE 6-PACK 11.2-13 OZ GLASS,Volume Share of Category,01-03-16,0.0710513875150094


In [152]:
temp["Value"] = temp["Value"].astype(float)
t = temp.pivot_table('Value', ["Product", "Date"], 'Measures')

t = t.reset_index().sort_values("Date")

brands = t.Product.unique()
final_df = pd.DataFrame()

for b in brands:
    k = t[t["Product"]==b].sort_values("Date")
    k['Ndate'] = ""
    count = 1
    year = 1
    for i, row in k.iterrows():
        #print(key, key_2)
        if year%5==0:
            year = 1
            count = (count)%52 + 1
        year += 1
        week_num = str(count)
        year_str = '20'+str(row['Date'].split("-")[-1])
        month_str = row['Date'].split("-")[0]
        if month_str[0] == '0':
            month_str = month_str[1]

        k.set_value(i, 'Ndate', " "+year_str+"-"+month_str+"-"+week_num+" ")
    
    final_df = pd.concat([final_df, k])

final_df.head(20)

Measures,Product,Date,Category Weighted Distribution,Price per Unit,Price per Volume,Unit Sales,Volume Sales,Volume Share of Category,Ndate
0,ABBEY DE LEFFE BLONDE PALE ALE 6-PACK 11.2-13 ...,01-03-13,36.890374,9.055060,38.812945,1211.340795,282.605807,0.048796,2013-1-1
1,ABBEY DE LEFFE BLONDE PALE ALE 6-PACK 11.2-13 ...,01-03-14,35.502502,9.131298,39.139726,1235.292070,288.193640,0.046605,2014-1-1
2,ABBEY DE LEFFE BLONDE PALE ALE 6-PACK 11.2-13 ...,01-03-15,45.154889,8.773517,37.606161,1660.126422,387.307494,0.056120,2015-1-1
3,ABBEY DE LEFFE BLONDE PALE ALE 6-PACK 11.2-13 ...,01-03-16,45.134618,9.427594,40.409746,1414.014578,329.889601,0.045482,2016-1-1
4,ABBEY DE LEFFE BLONDE PALE ALE 6-PACK 11.2-13 ...,01-10-13,37.821114,9.360824,40.123547,1150.765393,268.473566,0.061189,2013-1-2
5,ABBEY DE LEFFE BLONDE PALE ALE 6-PACK 11.2-13 ...,01-10-14,40.584528,8.959442,38.403094,1271.864017,296.725875,0.062644,2014-1-2
6,ABBEY DE LEFFE BLONDE PALE ALE 6-PACK 11.2-13 ...,01-10-15,43.813489,8.950018,38.362701,1229.132489,286.756610,0.064774,2015-1-2
7,ABBEY DE LEFFE BLONDE PALE ALE 6-PACK 11.2-13 ...,01-10-16,41.665073,9.087178,38.950613,1115.669208,260.285626,0.061719,2016-1-2
8,ABBEY DE LEFFE BLONDE PALE ALE 6-PACK 11.2-13 ...,01-17-13,38.447985,9.076937,38.906716,1392.074072,324.770881,0.065908,2013-1-3
9,ABBEY DE LEFFE BLONDE PALE ALE 6-PACK 11.2-13 ...,01-17-14,41.524211,9.061274,38.839578,1357.160200,316.625475,0.057424,2014-1-3


In [153]:
ext_df = pd.read_csv("external_data.csv")
ext_df.head(20)

,Ukey(YMW),Ukey(YM),Year,Month,Week,Quarter,Year-Quarter,Average of Max TemperatureC,Average of Mean TemperatureC,Average of Min TemperatureC,...,"Resident Population in Los Angeles County, CA, Thousands of Persons, Annual, Not Seasonally Adjusted","Per Capita Personal Income in Los Angeles County, CA, Dollars, Annual, Not Seasonally Adjusted",Occupancy (%) (Los Angeles County),Labor Force in LA,Employment in LA,Unemployment in LA,Unemp. Rate in LA,"Malt beverages, all types, all sizes, any origin, per 16 oz. (473.2 ml), US city average","Wine, red and white table, all sizes, any origin, per 1 liter (33.8 oz), US city average","Producer Price Index by Industry: Beer, Wine, and Liquor Stores: Retailing of Beer, Wine, and Liquor, Index Jun 2014=100, Monthly, Not Seasonally Adjusted, US level"
0,2013-1-1,2013-1,2013,1,1,Q1,2013Q1,19.00,12.20,5.00,...,10045.175,47580.0,69.631031,4945000.0,4439700.0,505300.0,10%,1.259,9.822,NaN
1,2013-1-2,2013-1,2013,1,2,Q1,2013Q1,17.43,12.00,6.71,...,10045.175,47580.0,69.631031,4945000.0,4439700.0,505300.0,10%,1.259,9.822,NaN
2,2013-1-3,2013-1,2013,1,3,Q1,2013Q1,20.43,12.86,5.00,...,10045.175,47580.0,69.631031,4945000.0,4439700.0,505300.0,10%,1.259,9.822,NaN
3,2013-1-4,2013-1,2013,1,4,Q1,2013Q1,23.57,17.57,11.43,...,10045.175,47580.0,69.631031,4945000.0,4439700.0,505300.0,10%,1.259,9.822,NaN
4,2013-1-5,2013-1,2013,1,5,Q1,2013Q1,19.20,14.00,8.40,...,10045.175,47580.0,69.631031,4945000.0,4439700.0,505300.0,10%,1.259,9.822,NaN
5,2013-10-40,2013-10,2013,10,40,Q4,2013Q4,27.80,22.00,16.20,...,10045.175,47580.0,78.210117,4992300.0,4531900.0,460400.0,9%,1.310,11.050,NaN
6,2013-10-41,2013-10,2013,10,41,Q4,2013Q4,24.71,19.43,13.71,...,10045.175,47580.0,78.210117,4992300.0,4531900.0,460400.0,9%,1.310,11.050,NaN
7,2013-10-42,2013-10,2013,10,42,Q4,2013Q4,28.14,21.14,14.00,...,10045.175,47580.0,78.210117,4992300.0,4531900.0,460400.0,9%,1.310,11.050,NaN
8,2013-10-43,2013-10,2013,10,43,Q4,2013Q4,22.57,18.43,13.86,...,10045.175,47580.0,78.210117,4992300.0,4531900.0,460400.0,9%,1.310,11.050,NaN
9,2013-10-44,2013-10,2013,10,44,Q4,2013Q4,22.20,17.20,12.40,...,10045.175,47580.0,78.210117,4992300.0,4531900.0,460400.0,9%,1.310,11.050,NaN


In [155]:
merged = pd.merge(final_df, ext_df, left_on="Ndate", right_on=" Ukey(YMW) ")
merged.to_csv("merged_ext_with_product.csv")
merged.shape

(76383, 50)

In [115]:
wo_marketing = pd.concat([merged, pd.DataFrame()])

marketing_data = pd.read_csv("features_data.csv")
marketing_data.head()

,Product,Causal,Measures,Week Ending 01-03-16,Week Ending 01-03-16.1,Week Ending 01-03-16.2,Week Ending 01-03-16.3,Week Ending 01-10-16,Week Ending 01-10-16.1,Week Ending 01-10-16.2,...,Week Ending 12-11-16.2,Week Ending 12-11-16.3,Week Ending 12-18-16,Week Ending 12-18-16.1,Week Ending 12-18-16.2,Week Ending 12-18-16.3,Week Ending 12-25-16,Week Ending 12-25-16.1,Week Ending 12-25-16.2,Week Ending 12-25-16.3
0,Product,Causal,Measures,Current,Year Ago,2 Years Ago,3 Years Ago,Current,Year Ago,2 Years Ago,...,2 Years Ago,3 Years Ago,Current,Year Ago,2 Years Ago,3 Years Ago,Current,Year Ago,2 Years Ago,3 Years Ago
1,BUD LIGHT 36-PACK 12 OZ CAN,Any Feature,Number of Merchandise Occurrences,280.50,274.68,273.39,284.74,183.97,274.60,272.29,...,274.61,274.39,269.65,280.94,274.52,274.22,269.58,280.55,NaN,273.69
2,BUD LIGHT 36-PACK 12 OZ CAN,Any Feature,Share of Merchandising Occurrences,0.76,0.70,1.09,1.57,0.55,1.12,0.97,...,0.97,0.90,0.78,0.82,0.84,0.86,0.68,0.73,NaN,0.74
3,BUD LIGHT 36-PACK 12 OZ CAN,Any Display,Number of Merchandise Occurrences,165.21,212.91,223.36,225.26,263.49,236.16,211.78,...,151.94,244.93,163.39,210.71,165.97,217.70,111.94,205.66,160.09,218.43
4,BUD LIGHT 36-PACK 12 OZ CAN,Any Display,Share of Merchandising Occurrences,0.95,1.19,1.71,1.87,1.69,1.56,1.55,...,1.11,2.04,1.09,1.20,1.00,1.55,0.65,1.12,1.00,1.51


In [138]:
cols = marketing_data.columns
weeks = marketing_data[3:]

new_df = pd.melt(marketing_data, id_vars=cols[:3], var_name="Date", value_name="Value")
new_df.head()

,Product,Causal,Measures,Date,Value
0,Product,Causal,Measures,Week Ending 01-03-16,Current
1,BUD LIGHT 36-PACK 12 OZ CAN,Any Feature,Number of Merchandise Occurrences,Week Ending 01-03-16,280.50
2,BUD LIGHT 36-PACK 12 OZ CAN,Any Feature,Share of Merchandising Occurrences,Week Ending 01-03-16,0.76
3,BUD LIGHT 36-PACK 12 OZ CAN,Any Display,Number of Merchandise Occurrences,Week Ending 01-03-16,165.21
4,BUD LIGHT 36-PACK 12 OZ CAN,Any Display,Share of Merchandising Occurrences,Week Ending 01-03-16,0.95


In [139]:
temp = pd.concat([new_df, pd.DataFrame()])
temp = temp.sort_values('Date')

temp = temp[temp["Measures"] != "Measures"]
temp.head()

for i, row in temp.iterrows():
    date = row["Date"]
    date = date.split(" ")[-1]
    if date.endswith("16.1"):
        date = date.replace("16.1", "15")
    if date.endswith("16.2"):
        date = date.replace("16.2", "14")
    if date.endswith("16.3"):
        date = date.replace("16.3", "13")
    temp.set_value(i, "Date", date)

print(temp.shape)
temp.head(-100)

(79040, 5)


,Product,Causal,Measures,Date,Value
259,BUD LIGHT 20-PACK 12 OZ CAN,Any Display,Number of Merchandise Occurrences,01-03-16,28.53
258,BUD LIGHT 20-PACK 12 OZ CAN,Any Feature,Share of Merchandising Occurrences,01-03-16,0.25
257,BUD LIGHT 20-PACK 12 OZ CAN,Any Feature,Number of Merchandise Occurrences,01-03-16,92.71
256,COORS 20-PACK 12 OZ SNBTL,Any Display,Share of Merchandising Occurrences,01-03-16,0.21
255,COORS 20-PACK 12 OZ SNBTL,Any Display,Number of Merchandise Occurrences,01-03-16,35.66
254,COORS 20-PACK 12 OZ SNBTL,Any Feature,Share of Merchandising Occurrences,01-03-16,0.70
253,COORS 20-PACK 12 OZ SNBTL,Any Feature,Number of Merchandise Occurrences,01-03-16,257.92
252,NEW BELGIUM FOLLY SAMPLER 12-PACK 11.2-13 OZ G...,Any Display,Share of Merchandising Occurrences,01-03-16,0.42
251,NEW BELGIUM FOLLY SAMPLER 12-PACK 11.2-13 OZ G...,Any Display,Number of Merchandise Occurrences,01-03-16,73.69
250,NEW BELGIUM FOLLY SAMPLER 12-PACK 11.2-13 OZ G...,Any Feature,Share of Merchandising Occurrences,01-03-16,1.21


In [143]:
temp["Value"] = temp["Value"].astype(float)
temp["Measures"] = temp["Causal"] + " " + temp["Measures"]

t = temp.pivot_table('Value', ["Product", "Date"], ['Causal','Measures'])

t = t.reset_index().sort_values("Date")

brands = t.Product.unique()
final_df = pd.DataFrame()

for b in brands:
    k = t[t["Product"]==b].sort_values("Date")
    k['Ndate'] = ""
    count = 1
    year = 1
    for i, row in k.iterrows():
        #print(key, key_2)
        if year%5==0:
            year = 1
            count = (count)%52 + 1
        year += 1
        week_num = str(count)
        year_str = '20'+str(row['Date'].split("-")[-1])
        month_str = row['Date'].split("-")[0]
        if month_str[0] == '0':
            month_str = month_str[1]

        k.set_value(i, 'Ndate', " "+year_str+"-"+month_str+"-"+week_num+" ")
    
    final_df = pd.concat([final_df, k])

final_df.head(20)

AttributeError: 'Series' object has no attribute 'split'

In [134]:
f_merged = pd.merge(final_df, merged, left_on=["Ndate", "Product"], right_on=["Ndate", "Product"])

In [135]:
f_merged.drop_duplicates().shape

(19665, 55)

In [126]:
f_merged.head()

,Product,Date_x,Number of Merchandise Occurrences,Share of Merchandising Occurrences,Ndate,Date_y,Category Weighted Distribution,Price per Unit,Price per Volume,Unit Sales,...,"Resident Population in Los Angeles County, CA, Thousands of Persons, Annual, Not Seasonally Adjusted","Per Capita Personal Income in Los Angeles County, CA, Dollars, Annual, Not Seasonally Adjusted",Occupancy (%) (Los Angeles County),Labor Force in LA,Employment in LA,Unemployment in LA,Unemp. Rate in LA,"Malt beverages, all types, all sizes, any origin, per 16 oz. (473.2 ml), US city average","Wine, red and white table, all sizes, any origin, per 1 liter (33.8 oz), US city average","Producer Price Index by Industry: Beer, Wine, and Liquor Stores: Retailing of Beer, Wine, and Liquor, Index Jun 2014=100, Monthly, Not Seasonally Adjusted, US level"
0,ANGRY ORCHARD CRISP APPLE 12-PACK 11.2-13 OZ G...,01-03-13,NaN,NaN,2013-1-1,01-03-13,8.673865,16.060000,32.126425,183.002185,...,10045.175,47580.0,69.631031,4945000.0,4439700.0,505300.0,10%,1.259,9.822,NaN
1,ANGRY ORCHARD CRISP APPLE 12-PACK 11.2-13 OZ G...,01-03-14,NaN,NaN,2014-1-1,01-03-14,1.482050,15.211923,30.429932,30.246917,...,10109.436,49400.0,73.600000,5004300.0,4562900.0,441300.0,9%,1.254,10.622,NaN
2,ANGRY ORCHARD CRISP APPLE 12-PACK 11.2-13 OZ G...,01-03-15,352.990,1.090,2015-1-1,01-03-15,74.875323,13.782617,27.570748,4561.858325,...,10170.292,NaN,71.600000,5033200.0,4650800.0,382400.0,8%,1.295,12.912,103.3
3,ANGRY ORCHARD CRISP APPLE 12-PACK 11.2-13 OZ G...,01-03-16,234.745,0.785,2016-1-1,01-03-16,77.516076,15.112090,30.230226,3969.933185,...,NaN,NaN,75.700000,4989100.0,4697800.0,291300.0,6%,1.356,12.180,98.9
4,ANGRY ORCHARD CRISP APPLE 12-PACK 11.2-13 OZ G...,01-10-13,NaN,NaN,2013-1-2,01-10-13,6.494534,16.048824,32.104068,99.117823,...,10045.175,47580.0,69.631031,4945000.0,4439700.0,505300.0,10%,1.259,9.822,NaN


In [127]:
f_merged.to_csv("final_merged.csv")